<a href="https://colab.research.google.com/github/cyuancheng/NLP_Yelp_review/blob/master/NLP_3_yelp_recom_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Recommendation System of Yelp Review using Keras

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
from fastai.text import *

In [84]:
import nltk #NLP data processing library (IMDB)
from nltk import word_tokenize, sent_tokenize
import gensim
from gensim.models.word2vec import Word2Vec
from sklearn.manifold import TSNE
import pandas as pd
from bokeh.io import output_notebook, output_file
from bokeh.plotting import show, figure

import string
from nltk.corpus import stopwords
from nltk.stem.porter import *
from gensim.models.phrases import Phraser, Phrases # phraser optimizes phrases
from keras.preprocessing.text import one_hot # relation

%matplotlib inline

Using TensorFlow backend.


In [49]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'School_of_AI/'

Mounted at /content/gdrive


In [53]:
df = pd.read_csv(base_dir+'export_review_rate.csv')
df.head()

,rate,review
0,4,I have been here at 630 am and the staff is al...
1,2,"Horrible experience this morning ,app orders s..."
2,1,I freqent this location practically every work...
3,3,I arrived extremely early in NY so I decided t...
4,3,"When I was younger, I used to play a game with..."


In [54]:
df['review'][1]

'Horrible experience this morning ,app orders screwed up. Got to work late because it toke them more than 15-20 minutes to get an order that takes 3 minutes...'

In [85]:
len(df)

146

In [92]:
nltk.download('stopwords')
stpwrds = stopwords.words('english') + list(string.punctuation) # words occur too much

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [128]:
print(stpwrds)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

preprocessing

In [0]:
df_texts = df.loc[0:146, 'review'].values
df_labels = df.loc[0:146, 'rate'].values

In [0]:
df_texts_clean = []
for i in range(146):
  lower_sents = []
  for word in df_texts[i].split():
    if not word.lower() in stpwrds:
      lower_sents.append(word.lower())
  df_texts_clean.append(lower_sents)

In [0]:
x_train = df_texts_clean[0:100]
y_train = df_labels[0:100]
x_valid = df_texts[101:146]
y_valid = df_labels[101:146]


In [162]:
print('# of Training Samples: {}'.format(len(x_train)))
print('# of validation Samples: {}'.format(len(x_valid )))
num_classes = max(y_train) + 1


# of Training Samples: 100
# of validation Samples: 45


In [0]:

#tokenizer = keras.preprocessing.text.Tokenizer(num_words=5000)
#x_train1 = tokenizer.fit_on_texts(x_train)

x_train1 = tokenizer.texts_to_matrix(x_train)


In [172]:
x_train1

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 0., 0.],
       [0., 1., 0.],
       [0., 0., 0.],
       [0., 0., 1.]])

In [168]:
from keras.preprocessing.text import Tokenizer
nb_words = 3
tokenizer = Tokenizer(nb_words=nb_words)



/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [169]:

tokenizer.fit_on_texts(["The sun is shining in June!","September is grey.","Life is beautiful in August.","I like it","This and other things?"])
print(tokenizer.word_index)

{'is': 1, 'in': 2, 'the': 3, 'sun': 4, 'shining': 5, 'june': 6, 'september': 7, 'grey': 8, 'life': 9, 'beautiful': 10, 'august': 11, 'i': 12, 'like': 13, 'it': 14, 'this': 15, 'and': 16, 'other': 17, 'things': 18}


In [0]:
# text classification
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential #model
from keras.layers import Dense, Flatten, Dropout
from keras.layers import Embedding # new! # feed data into neuron network
from keras.callbacks import ModelCheckpoint # new! # saving model
import os # new! 
from sklearn.metrics import roc_auc_score, roc_curve # new!
import pandas as pd
import matplotlib.pyplot as plt # new!
%matplotlib inline

In [0]:
# output directory name:
output_dir = 'model_output/dense'

# training:
epochs = 4
batch_size = 16

# vector-space embedding: 
n_dim = 64
n_unique_words = 100 # as per Maas et al. (2011); may not be optimal
n_words_to_skip = 50 # ditto
max_review_length = 100
pad_type = trunc_type = 'pre'

# neural network architecture: 
n_dense = 64
dropout = 0.5

In [72]:
df_val['text'][0]

"Gave them another chance today. Ordered a salad on the app. Came in and was told they don't have any.  Shouldn't the app and the store sync up for..."

In [60]:
df_trn['labels'].value_counts()

4    26
1    22
3    21
5    20
2    12
Name: labels, dtype: int64

In [61]:
df_val['labels'].value_counts()

1    13
5    10
3    10
4     7
2     5
Name: labels, dtype: int64

In [0]:
# Language model data
data_lm = TextLMDataBunch.from_df(base_dir , train_df=df_trn, valid_df=df_val)
# Classifier model data
data_clas = TextClasDataBunch.from_df(base_dir , train_df=df_trn, valid_df=df_val, vocab=data_lm.train_ds.vocab, bs=32)

In [74]:
data_clas.show_batch(1)

text,target
xxbos xxmaj xxunk than xxunk in xxmaj xxunk xxrep 4 . \n xxmaj however ! xxmaj xxunk at the xxunk and the barista were xxup so nice that i was like okay ... its okay if its slow . \n \n xxmaj xxunk . xxmaj xxunk of,4


In [0]:
#learn = language_model_learner(data_lm, AWD_LSTM, pretrained=URLs.WT103, drop_mult=0.3)
#learn = language_model_learner(data_lm, AWD_LSTM, pretrained_model=URLs.WT103_1, drop_mult=0.5)
#learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.1)
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [83]:
learn.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


KeyboardInterrupt: ignored

In [7]:
# Classifier model data, Tokenization
data_clas = TextClasDataBunch.from_csv(base_dir, 'export_review_rate.csv', vocab=data_lm.train_ds.vocab, bs=32)
data_clas.show_batch()

text,target
xxbos xxunk xxunk xxunk \n \n xxmaj this location is in the xxmaj xxunk and is xxup xxunk small . xxmaj the line go into the xxunk . xxmaj no place to really xxunk . \n \n xxmaj the line xxunk xxunk xxrep xxunk . \n \n xxmaj the staff,2
"xxbos xxmaj no bathroom ? ? ? xxmaj xxunk ! \n \n xxmaj should i pee on the floor after xxunk my two coffees ? \n \n xxmaj does the staff xxunk it in all day here ? \n \n xxmaj xxunk , it 's a decent xxmaj starbucks",3
xxbos xxmaj xxunk than xxunk in xxmaj xxunk xxrep 4 . \n xxmaj however ! xxmaj xxunk at the xxunk and the barista were xxup so nice that i was like xxunk ... xxunk xxunk if xxunk xxunk . \n \n xxmaj xxunk . xxmaj xxunk of,4
"xxbos xxmaj xxunk hours of xxunk , it xxunk at 4 xxunk on most days . \n \n i found the xxmaj xxunk to be fast and free ( xxup plus ) but they have xxup huge lines in the morning . xxmaj also , if",3
xxbos xxmaj have been to xxmaj starbucks all over the xxunk and never found a bad one until the one in xxmaj xxunk of xxmaj xxunk xxmaj xxunk xxmaj terminal in xxup nyc xxrep 4 . xxup the xxup worst . xxmaj xxunk with xxunk,1


In [8]:
data_clas.train_ds[0][0]

Text xxbos xxmaj coffee : i ordered my usual ( xxunk xxunk xxunk in a grande cup ) and it was xxunk . i 've come here many times and they always do it right . 
 
  xxmaj service : xxmaj the ...

In [9]:
data_clas.train_ds[0][0].data[:10]

array([  2,   5,  43, 177,  12,  75,  24, 219,  59,   0])

Data block API

In [0]:
bs = 32

In [0]:
data_lm = (TextList.from_df(df, cols=['label', 'text'])
                .random_split_by_pct(0.3)
                .label_for_lm()
                .databunch(bs=bs))


In [41]:
data_lm.show_batch()

idx,text
0,"the spot i xxunk xxrep 4 . xxbos 3 xxmaj when i was xxunk , i xxunk to xxunk a xxunk with my xxunk xxunk "" xxmaj would xxmaj you xxmaj xxunk "" . xxmaj as the xxunk so xxunk xxunk , one xxunk would xxunk two ... xxbos 1 xxmaj the wait here i just had for only a cup of ice is xxunk ! ! i normally"
1,way the xxunk ... xxbos 4 xxmaj this location is xxunk of xxunk away in the xxunk of the block . xxmaj it can get busy as xxunk a block away from xxmaj xxunk xxmaj xxunk . xxmaj there are some seats as xxunk ... xxbos 3 i xxunk here around xxup xxunk before work . xxmaj usually not many customers . xxmaj staff is xxunk very nice but
2,xxbos 1 xxmaj if i could give this place negative xxunk i would ! \n xxmaj my xxunk was a xxunk customer xxunk and they xxunk to xxunk xxunk use their xxunk ! xxbos 3 i stopped here while xxunk through the area . \n \n xxmaj service and drinks were just like i get at other xxmaj starbucks . xxmaj of xxunk there was a line .
3,"a xxmaj starbucks . i usually like to try the xxunk coffee xxunk but this one was conveniently located on the first xxunk of our ... xxbos 4 a xxunk - crowded place to stop for a quick coffee on xxunk . xxmaj there were xxunk seats , and only a xxunk of people came through the xxunk xxunk minutes we were there xxrep 4 . xxbos 1 xxmaj"
4,"and an xxunk to wait at the xxunk of the xxunk for my food , and xxunk minutes ... xxbos 2 xxmaj this xxmaj starbucks is located xxunk xxmaj the xxmaj xxunk on xxunk xxmaj ave . \n \n xxmaj there were two xxunk and they were xxunk to try and keep up with the line . xxmaj they could have ... xxbos 2 xxmaj one of the baristas"


In [42]:
np.shape(data_lm.train_ds)

(103, 2)

In [43]:
df.shape

(147, 2)

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, pretrained=URLs.WT103, drop_mult=0.3)

In [45]:
learn.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


KeyboardInterrupt: ignored

In [16]:
data_clas = (TextList.from_folder(base_dir, vocab=data_lm.vocab)
             #grab all the text files in path
             .split_by_rand_pct(0.1, seed = 42)
             .label_from_df(cols=0)
             #.split_by_folder(valid='test')
             #split by train and valid folder (that only keeps 'train' and 'test' so no need to filter)
             #.label_from_folder(classes=['neg', 'pos'])
             #label them all with their folders
             .databunch(bs=32))


/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:436: UserWarning: Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.
  warn("Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.")
/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:439: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


AttributeError: ignored